This notebook looks at the case when there is one UE connected to the base station, and then the xapp is started to log the rlc buffer change overtime. 

In [1]:
from fabric import Connection


tx = Connection(
    host='pc490.emulab.net',
    user = 'PeterYao',
    port=22,
)

delay = Connection(
    host='pc500.emulab.net',
    user = 'PeterYao',
    port=22,
)


router = Connection(
    host='pc487.emulab.net',
    user='PeterYao',
    port=22,
)

rx = Connection(
    host='pc816.emulab.net',
    user
    = 'PeterYao',  
    port=22,
)

conns = [router, delay, tx, rx]


In [ ]:
rx.sudo("killall iperf3")
rx.sudo("ip netns exec ue1 iperf3 -s -1 -p 4008 -D")
# run the client in the background
rx.run("iperf3 -c 12.1.1.130 -p 4008 -t 60 > /dev/null 2>&1 &", pty=False)

rx.run("/mydata/flexric/build/examples/xApp/c/monitor/xapp_gtp_mac_rlc_pdcp_moni")

In [2]:
rx.get("/tmp/xapp_db_1727528802524949")

In [3]:
import sqlite3
import pandas as pd

# Path to your SQLite file (replace 'your_database_file.sqlite' with your file)
db_file = 'xapp_db_1727528802524949'

# Connect to the SQLite database file
conn = sqlite3.connect(db_file)

# Fetch the list of tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", tables)

table_name = 'RLC_bearer'
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# Close the connection after extracting the data
conn.close()

# Display the DataFrame
print(df)

# # Save DataFrame to CSV if needed
df.to_csv('xapp_db_1727528802524949.csv', index=False)


Tables in the database:              name
0          MAC_UE
1      RLC_bearer
2     PDCP_bearer
3           SLICE
4        UE_SLICE
5        GTP_NGUT
6  KPM_MeasRecord
7   KPM_LabelInfo
                  tstamp  ngran_node  mcc  mnc  mnc_digit_len  nb_id cu_du_id  \
0       1727528804539588           2  208   95              2   3584   (null)   
1       1727528804539588           2  208   95              2   3584   (null)   
2       1727528804540585           2  208   95              2   3584   (null)   
3       1727528804540585           2  208   95              2   3584   (null)   
4       1727528804541586           2  208   95              2   3584   (null)   
...                  ...         ...  ...  ...            ...    ...      ...   
104281  1727528856706581           2  208   95              2   3584   (null)   
104282  1727528856707581           2  208   95              2   3584   (null)   
104283  1727528856707581           2  208   95              2   3584   (null)   
1042

: 

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Display the first few rows of the DataFrame
print(df.head())

# Extract 'tstamp' and 'txbuf_occ_bytes' columns
df_plot = df[['tstamp', 'txbuf_occ_bytes']]

# Adjust 'tstamp' to start at zero and convert to seconds
# Since 'tstamp' is in microseconds, divide by 1,000,000 to get seconds
df_plot['time_sec'] = (df_plot['tstamp'] - df_plot['tstamp'].min()) / 1_000_000

# Convert 'txbuf_occ_bytes' to Megabytes (MB)
df_plot['txbuf_occ_MB'] = df_plot['txbuf_occ_bytes'] / (1024 * 1024)

# Select data over the available time interval (up to 60 seconds)
df_plot_60s = df_plot[df_plot['time_sec'] <= 60]

# Plot 'txbuf_occ_MB' over 'time_sec'
plt.figure(figsize=(12, 6))
plt.plot(df_plot_60s['time_sec'], df_plot_60s['txbuf_occ_MB'])
plt.title('txbuf_occ_bytes Over Time (First 60 Seconds)')
plt.xlabel('Time (seconds)')
plt.ylabel('txbuf_occ_bytes (MB)')
plt.grid(True)
plt.show()
